In [1]:
# import modules
import pandas as pd
from env import host, user, password
import os
from sklearn.model_selection import train_test_split

In [2]:
# create function to get database url
# using function made in class
def get_url(db):
    '''
    This function takes in a database name and returns a url (using the specified 
    database name as well as host, user, and password from env.py) for use in the 
    pandas.read_sql() function.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [3]:
# create function to get telco dataframe and cache telco data to .csv file
# using function made in class
def acquire_telco():
    '''
    This function first searches for a .csv file containing the telco data and then 
    reads that file into a dataframe named telco. If telco.csv is not found, it uses
    the get_url() helper function to access the SQL server and read the query result 
    into a dataframe named telco. It then caches this data into a csv file (telco.csv).
    This function takes no arguments and returns the telco dataframe.
    '''
    if os.path.isfile('telco.csv'):
        telco = pd.read_csv('telco.csv', index_col=0)
        return telco
    else:
        sql = '''
        SELECT *
        FROM customers
        JOIN contract_types USING(contract_type_id)
        JOIN internet_service_types USING(internet_service_type_id)
        JOIN payment_types USING(payment_type_id)
        '''
        telco = pd.read_sql(sql, get_url('telco_churn'))
        telco.to_csv('telco.csv')
        return telco

In [4]:
# check that function works
telco = acquire_telco()
telco

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,3,0016-QLJIS,Female,0,Yes,Yes,65,Yes,...,Yes,Yes,Yes,Yes,90.45,5957.9,No,Two year,DSL,Mailed check
1,4,1,3,0017-DINOC,Male,0,No,No,54,No,...,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic)
2,3,1,3,0019-GFNTW,Female,0,No,No,56,No,...,Yes,No,No,No,45.05,2560.1,No,Two year,DSL,Bank transfer (automatic)
3,4,1,3,0056-EPFBG,Male,0,Yes,Yes,20,No,...,Yes,No,No,Yes,39.40,825.4,No,Two year,DSL,Credit card (automatic)
4,3,1,3,0078-XZMHT,Male,0,Yes,No,72,Yes,...,Yes,Yes,Yes,Yes,85.15,6316.2,No,Two year,DSL,Bank transfer (automatic)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2,3,1,9962-BFPDU,Female,0,Yes,Yes,1,Yes,...,No internet service,No internet service,No internet service,No,20.05,20.05,No,Month-to-month,None,Mailed check
7039,2,3,1,9967-ATRFS,Female,0,No,No,19,Yes,...,No internet service,No internet service,No internet service,No,19.90,367.55,No,Month-to-month,None,Mailed check
7040,4,3,1,9970-QBCDA,Female,0,No,No,6,Yes,...,No internet service,No internet service,No internet service,No,19.70,129.55,No,Month-to-month,None,Credit card (automatic)
7041,2,3,1,9975-SKRNR,Male,0,No,No,1,Yes,...,No internet service,No internet service,No internet service,No,18.90,18.9,No,Month-to-month,None,Mailed check


Now that the telco data has been assigned to a dataframe, I will clean, prepare, and split the data.

In [5]:
# check for null values
telco.isna().sum()

payment_type_id             0
internet_service_type_id    0
contract_type_id            0
customer_id                 0
gender                      0
senior_citizen              0
partner                     0
dependents                  0
tenure                      0
phone_service               0
multiple_lines              0
online_security             0
online_backup               0
device_protection           0
tech_support                0
streaming_tv                0
streaming_movies            0
paperless_billing           0
monthly_charges             0
total_charges               0
churn                       0
contract_type               0
internet_service_type       0
payment_type                0
dtype: int64

There appear to be no null values that need to be dropped.

When I retrieved this data from the mySQL server, I joined multiple tables to create this dataframe. We no longer need the foreign key columns ('payment_type_id', 'internet_service_type_id', and 'contract_type_id') used to join the tables since they have corresponding columns ('payment_type', 'internet_service_type', and 'contract_type') essentially providing the same data, so I decided to drop the foreign key columns.

The 'customer_id' column doesn't contain data that will be useful for the purpose of finding drivers of churn, but it will be necessary for the .csv file I will create later in this project. However, I decided to drop 'customer_id' at this point to make cleaning the data somewhat easier. I will add this column back to the dataframe before moving onto splitting the data.

In [6]:
# drop foreign key columns, customer_id, and any duplicate columns
telco = telco.drop_duplicates()
telco = telco.drop(columns=['payment_type_id', 'internet_service_type_id', 'contract_type_id', 'customer_id'])
telco

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,Female,0,Yes,Yes,65,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,90.45,5957.9,No,Two year,DSL,Mailed check
1,Male,0,No,No,54,No,No phone service,Yes,No,No,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic)
2,Female,0,No,No,56,No,No phone service,Yes,Yes,Yes,Yes,No,No,No,45.05,2560.1,No,Two year,DSL,Bank transfer (automatic)
3,Male,0,Yes,Yes,20,No,No phone service,Yes,No,Yes,Yes,No,No,Yes,39.40,825.4,No,Two year,DSL,Credit card (automatic)
4,Male,0,Yes,No,72,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,85.15,6316.2,No,Two year,DSL,Bank transfer (automatic)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Female,0,Yes,Yes,1,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.05,20.05,No,Month-to-month,None,Mailed check
7039,Female,0,No,No,19,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,19.90,367.55,No,Month-to-month,None,Mailed check
7040,Female,0,No,No,6,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,19.70,129.55,No,Month-to-month,None,Credit card (automatic)
7041,Male,0,No,No,1,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,18.90,18.9,No,Month-to-month,None,Mailed check


In [7]:
# look at dataframe info
telco.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   gender                 7043 non-null   object 
 1   senior_citizen         7043 non-null   int64  
 2   partner                7043 non-null   object 
 3   dependents             7043 non-null   object 
 4   tenure                 7043 non-null   int64  
 5   phone_service          7043 non-null   object 
 6   multiple_lines         7043 non-null   object 
 7   online_security        7043 non-null   object 
 8   online_backup          7043 non-null   object 
 9   device_protection      7043 non-null   object 
 10  tech_support           7043 non-null   object 
 11  streaming_tv           7043 non-null   object 
 12  streaming_movies       7043 non-null   object 
 13  paperless_billing      7043 non-null   object 
 14  monthly_charges        7043 non-null   float64
 15  tota

The column 'total_charges' is an object type, but needs to be a float for our analysis and modeling. I will look deeper into why it is not a numeric dtype.

In [8]:
# look at values in total_charges column
telco.total_charges.value_counts()

           11
20.2       11
19.75       9
20.05       8
19.65       8
           ..
1772.25     1
3088.75     1
3207.55     1
3186.7      1
5574.75     1
Name: total_charges, Length: 6531, dtype: int64

In [9]:
# see what ' ' corresponds to
telco[telco.total_charges == ' ']

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
85,Female,0,Yes,Yes,0,No,No phone service,Yes,Yes,Yes,Yes,Yes,No,No,56.05,,No,Two year,DSL,Credit card (automatic)
156,Male,0,No,Yes,0,Yes,Yes,Yes,Yes,No,Yes,No,No,Yes,61.90,,No,Two year,DSL,Bank transfer (automatic)
236,Female,0,Yes,Yes,0,Yes,Yes,No,Yes,Yes,Yes,Yes,No,No,73.35,,No,Two year,DSL,Mailed check
255,Female,0,Yes,Yes,0,No,No phone service,Yes,No,Yes,Yes,Yes,No,Yes,52.55,,No,Two year,DSL,Bank transfer (automatic)
339,Female,0,Yes,Yes,0,Yes,No,Yes,Yes,Yes,No,Yes,Yes,No,80.85,,No,Two year,DSL,Mailed check
5681,Female,0,Yes,Yes,0,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.00,,No,Two year,None,Mailed check
5717,Male,0,No,Yes,0,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.25,,No,Two year,None,Mailed check
5727,Male,0,Yes,Yes,0,Yes,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,25.35,,No,Two year,None,Mailed check
5798,Male,0,Yes,Yes,0,Yes,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,25.75,,No,Two year,None,Mailed check
6007,Male,0,Yes,Yes,0,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,19.85,,No,Two year,None,Mailed check


All rows with total_charges of ' ' have a tenure of 0. It seems they have no total_charges because they are new customers who have not yet been charged. To be sure, I'll check for any other rows with a tenure of 0.

In [10]:
# see if any other rows have a 0 tenure
telco[telco.tenure == 0]

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
85,Female,0,Yes,Yes,0,No,No phone service,Yes,Yes,Yes,Yes,Yes,No,No,56.05,,No,Two year,DSL,Credit card (automatic)
156,Male,0,No,Yes,0,Yes,Yes,Yes,Yes,No,Yes,No,No,Yes,61.90,,No,Two year,DSL,Bank transfer (automatic)
236,Female,0,Yes,Yes,0,Yes,Yes,No,Yes,Yes,Yes,Yes,No,No,73.35,,No,Two year,DSL,Mailed check
255,Female,0,Yes,Yes,0,No,No phone service,Yes,No,Yes,Yes,Yes,No,Yes,52.55,,No,Two year,DSL,Bank transfer (automatic)
339,Female,0,Yes,Yes,0,Yes,No,Yes,Yes,Yes,No,Yes,Yes,No,80.85,,No,Two year,DSL,Mailed check
5681,Female,0,Yes,Yes,0,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.00,,No,Two year,None,Mailed check
5717,Male,0,No,Yes,0,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.25,,No,Two year,None,Mailed check
5727,Male,0,Yes,Yes,0,Yes,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,25.35,,No,Two year,None,Mailed check
5798,Male,0,Yes,Yes,0,Yes,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,25.75,,No,Two year,None,Mailed check
6007,Male,0,Yes,Yes,0,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,19.85,,No,Two year,None,Mailed check


The only rows with a tenure of 0 are those with total_charges of ' '. Since these are new customers who have not yet had the opportunity to churn, their data will probably not be useful in helping find a driver of churn. Therefore, I will drop the 11 rows with total_charges of ' ' and change total_charges to a float dtype.

In [11]:
# drop rows where total_charges == ' '
telco = telco[telco.total_charges != ' ']
# change total_charges to float dtype
telco.total_charges = telco.total_charges.astype(float)

/opt/homebrew/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [12]:
# confirm that dtype is now correct
telco.dtypes

gender                    object
senior_citizen             int64
partner                   object
dependents                object
tenure                     int64
phone_service             object
multiple_lines            object
online_security           object
online_backup             object
device_protection         object
tech_support              object
streaming_tv              object
streaming_movies          object
paperless_billing         object
monthly_charges          float64
total_charges            float64
churn                     object
contract_type             object
internet_service_type     object
payment_type              object
dtype: object

Next, I will create dummy variables for categorical variables so that I can use these variables in the models I will create later in this project.

In [13]:
# view value counts for each categorical variable
for col in list(telco.select_dtypes('O').columns):
    print(telco[col].value_counts())
    print()

Male      3549
Female    3483
Name: gender, dtype: int64

No     3639
Yes    3393
Name: partner, dtype: int64

No     4933
Yes    2099
Name: dependents, dtype: int64

Yes    6352
No      680
Name: phone_service, dtype: int64

No                  3385
Yes                 2967
No phone service     680
Name: multiple_lines, dtype: int64

No                     3497
Yes                    2015
No internet service    1520
Name: online_security, dtype: int64

No                     3087
Yes                    2425
No internet service    1520
Name: online_backup, dtype: int64

No                     3094
Yes                    2418
No internet service    1520
Name: device_protection, dtype: int64

No                     3472
Yes                    2040
No internet service    1520
Name: tech_support, dtype: int64

No                     2809
Yes                    2703
No internet service    1520
Name: streaming_tv, dtype: int64

No                     2781
Yes                    2731
No inter

In [14]:
# create dataframe of dummy variables
dummies = pd.get_dummies(telco[telco.select_dtypes('O').columns], drop_first=True)
dummies.head()

,gender_Male,partner_Yes,dependents_Yes,phone_service_Yes,multiple_lines_No phone service,multiple_lines_Yes,online_security_No internet service,online_security_Yes,online_backup_No internet service,online_backup_Yes,...,streaming_movies_Yes,paperless_billing_Yes,churn_Yes,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,0,1,1,1,0,1,0,1,0,1,...,1,1,0,0,1,0,0,0,0,1
1,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
2,0,0,0,0,1,0,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0
3,1,1,1,0,1,0,0,1,0,0,...,0,1,0,0,1,0,0,1,0,0
4,1,1,0,1,0,1,0,0,0,1,...,1,1,0,0,1,0,0,0,0,0


In [15]:
# concatenate dummy dataframe and telco dataframe
telco = pd.concat([telco, dummies], axis=1)
telco.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,...,streaming_movies_Yes,paperless_billing_Yes,churn_Yes,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,Female,0,Yes,Yes,65,Yes,Yes,Yes,Yes,Yes,...,1,1,0,0,1,0,0,0,0,1
1,Male,0,No,No,54,No,No phone service,Yes,No,No,...,0,0,0,0,1,0,0,1,0,0
2,Female,0,No,No,56,No,No phone service,Yes,Yes,Yes,...,0,0,0,0,1,0,0,0,0,0
3,Male,0,Yes,Yes,20,No,No phone service,Yes,No,Yes,...,0,1,0,0,1,0,0,1,0,0
4,Male,0,Yes,No,72,Yes,Yes,No,Yes,Yes,...,1,1,0,0,1,0,0,0,0,0


In [16]:
# drop columns we now have dummies for
telco.drop(columns=telco.select_dtypes('O').columns, inplace=True)

In [17]:
# check columns
telco.columns

Index(['senior_citizen', 'tenure', 'monthly_charges', 'total_charges',
       'gender_Male', 'partner_Yes', 'dependents_Yes', 'phone_service_Yes',
       'multiple_lines_No phone service', 'multiple_lines_Yes',
       'online_security_No internet service', 'online_security_Yes',
       'online_backup_No internet service', 'online_backup_Yes',
       'device_protection_No internet service', 'device_protection_Yes',
       'tech_support_No internet service', 'tech_support_Yes',
       'streaming_tv_No internet service', 'streaming_tv_Yes',
       'streaming_movies_No internet service', 'streaming_movies_Yes',
       'paperless_billing_Yes', 'churn_Yes', 'contract_type_One year',
       'contract_type_Two year', 'internet_service_type_Fiber optic',
       'internet_service_type_None', 'payment_type_Credit card (automatic)',
       'payment_type_Electronic check', 'payment_type_Mailed check'],
      dtype='object')

In [18]:
# rename columns for clarity/ease of use
telco.rename(columns={'gender_Male':'is_male',
                      'partner_Yes':'has_partner',
                      'dependents_Yes':'has_dependents',
                      'phone_service_Yes':'has_phone_service',
                      'paperless_billing_Yes':'has_paperless_billing',
                      'churn_Yes':'has_churned'}, inplace=True)
telco.head()

,senior_citizen,tenure,monthly_charges,total_charges,is_male,has_partner,has_dependents,has_phone_service,multiple_lines_No phone service,multiple_lines_Yes,...,streaming_movies_Yes,has_paperless_billing,has_churned,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,0,65,90.45,5957.90,0,1,1,1,0,1,...,1,1,0,0,1,0,0,0,0,1
1,0,54,45.20,2460.55,1,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,0
2,0,56,45.05,2560.10,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3,0,20,39.40,825.40,1,1,1,0,1,0,...,0,1,0,0,1,0,0,1,0,0
4,0,72,85.15,6316.20,1,1,0,1,0,1,...,1,1,0,0,1,0,0,0,0,0


In [19]:
# add 'customer_id' column back in for remaining rows
telco['customer_id'] = acquire_telco()[acquire_telco().total_charges != ' '].customer_id
telco.head()

,senior_citizen,tenure,monthly_charges,total_charges,is_male,has_partner,has_dependents,has_phone_service,multiple_lines_No phone service,multiple_lines_Yes,...,has_paperless_billing,has_churned,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check,customer_id
0,0,65,90.45,5957.90,0,1,1,1,0,1,...,1,0,0,1,0,0,0,0,1,0016-QLJIS
1,0,54,45.20,2460.55,1,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0017-DINOC
2,0,56,45.05,2560.10,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0019-GFNTW
3,0,20,39.40,825.40,1,1,1,0,1,0,...,1,0,0,1,0,0,1,0,0,0056-EPFBG
4,0,72,85.15,6316.20,1,1,0,1,0,1,...,1,0,0,1,0,0,0,0,0,0078-XZMHT


In [20]:
# combine previous steps into single function
def clean_telco(telco):
    '''
    This function takes in the telco dataframe and returns a clean version of the 
    dataframe. It drops unnecessary or duplicate columns and rows, prepares the data
    to work with machine learning models, and renames columns for clarity.
    '''
    telco = telco.drop_duplicates()
    telco = telco.drop(columns=['payment_type_id', 'internet_service_type_id', 'contract_type_id', 'customer_id'])
    telco = telco[telco.total_charges != ' ']
    telco.total_charges = telco.total_charges.astype(float)
    dummies = pd.get_dummies(telco[telco.select_dtypes('O').columns], drop_first=True)
    telco = pd.concat([telco, dummies], axis=1)
    telco.drop(columns=telco.select_dtypes('O').columns, inplace=True)
    telco.rename(columns={'gender_Male':'is_male',
                      'partner_Yes':'has_partner',
                      'dependents_Yes':'has_dependents',
                      'phone_service_Yes':'has_phone_service',
                      'paperless_billing_Yes':'has_paperless_billing',
                      'churn_Yes':'has_churned'}, inplace=True)
    telco['customer_id'] = acquire_telco()[acquire_telco().total_charges != ' '].customer_id
    return telco

Now that I have cleaned the telco data and created a function to perform all of the steps I went through, I will test the function by retrieving a fresh copy of the uncleaned data and then running the clean_telco() function to confirm its results.

In [21]:
# assign unprepped data to variable
telco = acquire_telco()
# confirm function results are correct
telco = clean_telco(telco)
telco.head()

,senior_citizen,tenure,monthly_charges,total_charges,is_male,has_partner,has_dependents,has_phone_service,multiple_lines_No phone service,multiple_lines_Yes,...,has_paperless_billing,has_churned,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check,customer_id
0,0,65,90.45,5957.90,0,1,1,1,0,1,...,1,0,0,1,0,0,0,0,1,0016-QLJIS
1,0,54,45.20,2460.55,1,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0017-DINOC
2,0,56,45.05,2560.10,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0019-GFNTW
3,0,20,39.40,825.40,1,1,1,0,1,0,...,1,0,0,1,0,0,1,0,0,0056-EPFBG
4,0,72,85.15,6316.20,1,1,0,1,0,1,...,1,0,0,1,0,0,0,0,0,0078-XZMHT


With the knowledge that my acquire and cleaning functions work, I will move on to the next step in the data preparation process: splitting the data. For this, I will use the function we defined in class. I will be splitting the data into three samples: train (56%), validate (24%), and test (20%). I will stratify based on our target variable (has_churned) to ensure an even distribution among each sample. I will also set the random_state to a seed (123) so that my results can be reproduced.

In [22]:
# create function to split telco data into three samples
def split_telco(telco):
    '''
    This function takes in the telco dataframe and splits it into three dataframes.
    It returns these dataframes in this order: train, validate, test.
    '''
    train, test = train_test_split(telco, test_size=0.2, random_state=123, stratify=telco.has_churned)
    train, validate = train_test_split(train, test_size=0.3, random_state=123, stratify=train.has_churned)
    return train, validate, test

In [23]:
# check that function works
train, validate, test = split_telco(telco)
train.shape, validate.shape, test.shape

((3937, 32), (1688, 32), (1407, 32))

Now that I know the split function works, I can combine all of my acquisition and preparation functions into a single wrangling function.

In [24]:
# combine prep functions into single wrangle function
def wrangle_telco():
    '''
    This function combines the acquire_telco(), clean_telco() and split_telco() functions.
    It takes no arguments, cleans the telco data, splits it, and returns three dataframes 
    called train, validate, and test.
    '''
    telco = acquire_telco()
    telco = clean_telco(telco)
    train, validate, test = split_telco(telco)
    return train, validate, test

In [25]:
# ensure wrangle function works
train, validate, test = wrangle_telco()
print(train.shape, validate.shape, test.shape)
train.head()

(3937, 32) (1688, 32) (1407, 32)


,senior_citizen,tenure,monthly_charges,total_charges,is_male,has_partner,has_dependents,has_phone_service,multiple_lines_No phone service,multiple_lines_Yes,...,has_paperless_billing,has_churned,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check,customer_id
6096,0,70,19.45,1303.50,0,1,0,1,0,0,...,0,0,0,1,0,1,0,0,0,8966-SNIZF
1603,0,15,69.50,1071.40,1,1,1,1,0,0,...,1,0,0,0,0,0,0,0,1,3466-BYAVD
5350,1,52,109.10,5647.95,0,1,0,1,0,1,...,1,0,0,0,1,0,0,1,0,9278-VZKCD
2068,0,39,40.60,1494.50,1,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,7074-STDCN
6366,0,32,19.30,593.20,0,1,0,1,0,0,...,0,0,1,0,0,1,0,0,1,5914-GXMDA
